# Changelog

All notable changes to this project will be documented in this file.

## [0.1]

### DIRECTORY
- dataset_wall_corrected_2

### CHANGES:
- remove walls' extensions 
- manual clear over exported dataset has been done

## [0.2] - 2023-11-22

### DIRECTORY
- dataset_fine_1

### CHANGES:
- export only:
    - with ortho walls
    - when entrance is beside communication space
    - there are all room types in unit
    - takes into consideration manual clear of previous dataset

## [0.3] - 2023-12-30

### DIRECTORY
- dataset_fine_256

### CHANGES:
- size: 256 x 256

## Data source:
Matthias Standfest, Michael Franzen, Yvonne Schröder, Luis Gonzalez Medina, Yarilo Villanueva Hernandez, Jan Hendrik Buck, Yen-Ling Tan, Milena Niedzwiecka, & Rachele Colmegna, Swiss Dwellings: A large dataset of apartment models including aggregated geolocation-based simulation results covering viewshed, natural light, traffic noise, centrality and geometric analysis (3.0.0) [Data set], Zenodo, 2022, URL: https://zenodo.org/records/7788422

License:  Creative Commons Attribution 4.0 International 

In [28]:
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image, ImageDraw
import shapely.affinity
from csv import DictReader
import shapely
from sortedcontainers import SortedList, SortedKeyList

In [29]:
with open('geometries.csv', mode='r') as infile:
   
    dict_reader = DictReader(infile)     
    list_of_dict = list(dict_reader)

In [30]:
list_of_dict[0]

{'apartment_id': '3c3b1d6ca8b4b9092480b8c75f9eaa81',
 'site_id': '210',
 'building_id': '399',
 'plan_id': '1054',
 'floor_id': '1588',
 'unit_id': '7300',
 'area_id': '619311.0',
 'unit_usage': 'RESIDENTIAL',
 'entity_type': 'area',
 'entity_subtype': 'BATHROOM',
 'geometry': 'POLYGON ((-2.7337844078265210 4.0798074408321536, -1.7052982090605919 5.3055095631450140, -1.1780751324833911 4.8631168740409318, -1.2821478117691409 4.7390878846112354, 0.0038369733703361 3.6600185257011066, -0.9205765461098432 2.5583453928179463, -2.7337844078265210 4.0798074408321536))',
 'elevation': '0.0',
 'height': '2.6'}

In [31]:
for e in list_of_dict:
    e["polygon"] = shapely.from_wkt(e["geometry"], on_invalid='warn')

In [32]:
apartments_id = SortedList()
sorted_list_of_dict = SortedKeyList(key = lambda d : d['apartment_id']) # key list

for e in list_of_dict:
    if e['unit_usage'] == 'RESIDENTIAL':
        sorted_list_of_dict.add(e)

for e in sorted_list_of_dict:
    if e["apartment_id"] not in apartments_id:
        apartments_id.add(e["apartment_id"])

In [33]:
len(apartments_id)

44894

In [34]:
# entity_subtype = SortedList()

# for e in list_of_dict:
#     if e["entity_subtype"] not in entity_subtype:
#         entity_subtype.add(e["entity_subtype"])

In [35]:
# entity_type = SortedList()

# for e in list_of_dict:
#     if e["entity_type"] not in entity_type:
#         entity_type.add(e["entity_type"])

In [36]:
colors_out = {
    'wall': (0, 0, 0),
    'sanitary': (0, 255, 0), 
    'priv_space':  (255, 0, 255),
    'kitchen': (0, 255, 128),
    'comunication': (0, 128, 255),
    'public_space':  (255, 255, 0),
    'other_space': (128, 128, 128),
    'dining': (0, 255, 128), # like kitchen
    'windows': (0, 0, 255),
    'main_door': (255, 0, 0),
    'internal_doors': (0, 255, 255),
    'balcony': (128, 128, 128)
    #'test': (255, 140, 0)
}

colors_in = {
    'wall': (0, 0, 0),
    'sanitary': (0, 0, 0),
    'priv_space':  (0, 0, 0),
    'kitchen': (0, 0, 0),
    'comunication': (0, 0, 0),
    'public_space':  (0, 0, 0),
    'other_space': (0, 0, 0),
    'dining': (0, 0, 0), # like kitchen
    'windows': (0, 0, 255),
    'main_door': (255, 0, 0),
    'internal_doors': (0, 0, 0),
    'balcony': (128, 128, 128)
    #'test': (255, 140, 0)

}


In [37]:
output_display = {
    'AIR': None,
    'ARCADE': None,
    'ARCHIVE': colors_out['other_space'],
    'BALCONY': colors_out['balcony'],
    'BASEMENT': colors_out['other_space'],
    'BASEMENT_COMPARTMENT': colors_out['other_space'],
    'BATHROOM': colors_out['sanitary'],
    'BATHTUB': None,
    'BEDROOM': colors_out['priv_space'],
    'BIKE_STORAGE': colors_out['other_space'],
    'BREAK_ROOM':  colors_out['other_space'],
    'BUILT_IN_FURNITURE': None,
    'CANTEEN': colors_out['other_space'],
    'CARPARK': colors_out['other_space'],
    'CLOAKROOM': colors_out['other_space'],
    'COLD_STORAGE': colors_out['other_space'],
    'COLUMN': colors_out['wall'],
    'COMMON_KITCHEN': colors_out['kitchen'],
    'COMMUNITY_ROOM': colors_out["public_space"],
    'CORRIDOR': colors_out["comunication"],
    'CORRIDORS_AND_HALLS': colors_out["comunication"],
    'COUNTER_ROOM': colors_out['other_space'],
    'DEDICATED_MEDICAL_ROOM': colors_out['other_space'],
    'DINING': colors_out['dining'],
    'DOOR': colors_out['internal_doors'],
    'ELECTRICAL_SUPPLY': None,
    'ELEVATOR': colors_out['comunication'],
    'ELEVATOR_FACILITIES': None,
    'ENTRANCE_DOOR': colors_out["main_door"],
    'FACTORY_ROOM': colors_out['other_space'],
    'FOYER': colors_out["comunication"],
    'GARAGE': colors_out['other_space'],
    'GARDEN': None,
    'GAS': None,
    'HEATING': None,
    'HOUSE_TECHNICS_FACILITIES': None,
    'KITCHEN': colors_out["kitchen"],
    'KITCHEN_DINING': colors_out["kitchen"],
    'LIGHTWELL': None,
    'LIVING_DINING': colors_out['public_space'],
    'LIVING_ROOM': colors_out['public_space'],
    'LOBBY': colors_out['public_space'],
    'LOGGIA': colors_out['balcony'],
    'LOGISTICS': colors_out['other_space'],
    'MEDICAL_ROOM': colors_out['other_space'],
    'MEETING_ROOM': colors_out['other_space'],
    'NOT_DEFINED': colors_out['other_space'],
    'OFFICE': colors_out['other_space'],
    'OFFICE_SPACE': colors_out['other_space'],
    'OFFICE_TECH_ROOM': colors_out['other_space'],
    'OIL_TANK': None,
    'OPEN_PLAN_OFFICE': colors_out['other_space'],
    'OPERATIONS_FACILITIES': None,
    'OUTDOOR_VOID': None,
    'PATIO': None,
    'PHYSIO_AND_REHABILITATION': colors_out['other_space'],
    'PRAM': colors_out['other_space'],
    'PRAM_AND_BIKE_STORAGE_ROOM': colors_out['other_space'],
    'RADATION_THERAPY': colors_out['other_space'],
    'RAILING': None,
    'RAMP': colors_out['comunication'],
    'RECEPTION_ROOM': colors_out['other_space'],
    'ROOM': colors_out['priv_space'],
    'SALESROOM': colors_out['other_space'],
    'SANITARY_ROOMS': colors_out['sanitary'],
    'SHAFT': None,
    'SHELTER': colors_out['other_space'],
    'SHOWER': None,
    'SHOWROOM': colors_out['other_space'],
    'SINK': None,
    'SPORTS_ROOMS': colors_out['other_space'],
    'STAIRCASE': colors_out['comunication'],
    'STAIRS': colors_out['comunication'],
    'STOREROOM': colors_out['other_space'],
    'STUDIO': colors_out['other_space'],
    'TEACHING_ROOM': colors_out['other_space'],
    'TECHNICAL_AREA': colors_out['other_space'],
    'TERRACE': colors_out['other_space'],
    'TOILET': colors_out['sanitary'],
    'TRANSPORT_SHAFT': None,
    'VEHICLE_TRAFFIC_AREA': None,
    'VOID': None,
    'WAITING_ROOM': colors_out['other_space'],
    'WALL': colors_out['wall'],
    'WAREHOUSE': colors_out['other_space'],
    'WASHING_MACHINE': None,
    'WASH_AND_DRY_ROOM': colors_out['other_space'],
    'WATER_SUPPLY': None,
    'WINDOW': colors_out['windows'],
    'WINTERGARTEN': colors_out['public_space'],
    'WORKSHOP': colors_out['other_space']
}

input_display = {
    'AIR': None,
    'ARCADE': None,
    'ARCHIVE': colors_in['other_space'],
    'BALCONY': colors_in['balcony'], #colors_in['other_space'],
    'BASEMENT': colors_in['other_space'],
    'BASEMENT_COMPARTMENT': colors_in['other_space'],
    'BATHROOM': colors_in['sanitary'],
    'BATHTUB': None,
    'BEDROOM': colors_in['priv_space'],
    'BIKE_STORAGE': colors_in['other_space'],
    'BREAK_ROOM':  colors_in['other_space'],
    'BUILT_IN_FURNITURE': None,
    'CANTEEN': colors_in['other_space'],
    'CARPARK': colors_in['other_space'],
    'CLOAKROOM': colors_in['other_space'],
    'COLD_STORAGE': colors_in['other_space'],
    'COLUMN': colors_in['wall'],
    'COMMON_KITCHEN': colors_in['kitchen'],
    'COMMUNITY_ROOM': colors_in['public_space'],
    'CORRIDOR': colors_in['comunication'],
    'CORRIDORS_AND_HALLS': colors_in['comunication'],
    'COUNTER_ROOM': colors_in['other_space'],
    'DEDICATED_MEDICAL_ROOM': colors_in['other_space'],
    'DINING': colors_in['dining'],
    'DOOR': colors_in['internal_doors'],
    'ELECTRICAL_SUPPLY': None,
    'ELEVATOR': colors_in['comunication'],
    'ELEVATOR_FACILITIES': None,
    'ENTRANCE_DOOR': colors_in['main_door'],
    'FACTORY_ROOM': colors_in['other_space'],
    'FOYER': colors_in['comunication'],
    'GARAGE': colors_in['other_space'],
    'GARDEN': None,
    'GAS': None,
    'HEATING': None,
    'HOUSE_TECHNICS_FACILITIES': None,
    'KITCHEN': colors_in['kitchen'],
    'KITCHEN_DINING': colors_in['kitchen'],
    'LIGHTWELL': None,
    'LIVING_DINING': colors_in['public_space'],
    'LIVING_ROOM': colors_in['public_space'],
    'LOBBY': colors_in['public_space'],
    'LOGGIA': colors_in['balcony'], #colors_in['other_space'], 
    'LOGISTICS': colors_in['other_space'],
    'MEDICAL_ROOM': colors_in['other_space'],
    'MEETING_ROOM': colors_in['other_space'],
    'NOT_DEFINED': colors_in['other_space'],
    'OFFICE': colors_in['other_space'],
    'OFFICE_SPACE': colors_in['other_space'],
    'OFFICE_TECH_ROOM': colors_in['other_space'],
    'OIL_TANK': None,
    'OPEN_PLAN_OFFICE': colors_in['other_space'],
    'OPERATIONS_FACILITIES': None,
    'OUTDOOR_VOID': None, # ??
    'PATIO': None,
    'PHYSIO_AND_REHABILITATION': colors_in['other_space'],
    'PRAM': colors_in['other_space'],
    'PRAM_AND_BIKE_STORAGE_ROOM': colors_in['other_space'],
    'RADATION_THERAPY': colors_in['other_space'],
    'RAILING': None,
    'RAMP': colors_in['comunication'],
    'RECEPTION_ROOM': colors_in['other_space'],
    'ROOM': colors_in['priv_space'],
    'SALESROOM': colors_in['other_space'],
    'SANITARY_ROOMS': colors_in['sanitary'],
    'SHAFT': colors_in['other_space'],
    'SHELTER': colors_in['other_space'],
    'SHOWER': None,
    'SHOWROOM': colors_in['other_space'],
    'SINK': None,
    'SPORTS_ROOMS': colors_in['other_space'],
    'STAIRCASE': colors_in['comunication'],
    'STAIRS': colors_in['comunication'],
    'STOREROOM': colors_in['other_space'],
    'STUDIO': colors_in['other_space'],
    'TEACHING_ROOM': colors_in['other_space'],
    'TECHNICAL_AREA': colors_in['other_space'],
    'TERRACE': None,
    'TOILET': colors_in['sanitary'],
    'TRANSPORT_SHAFT': colors_in['other_space'],
    'VEHICLE_TRAFFIC_AREA': colors_in['other_space'],
    'VOID': colors_in['other_space'],
    'WAITING_ROOM': colors_in['other_space'],
    'WALL': colors_in['wall'],
    'WAREHOUSE': colors_in['other_space'],
    'WASHING_MACHINE': None,
    'WASH_AND_DRY_ROOM': colors_in['other_space'],
    'WATER_SUPPLY': None,
    'WINDOW': colors_in['windows'],
    'WINTERGARTEN': colors_in['public_space'],
    'WORKSHOP': colors_in['other_space']
}

In [38]:
list_of_apartments = list()  # list of lists containing apartments' geometry

for id in apartments_id:
    if id != "":
        apartment_all_geom = list()
        for e in sorted_list_of_dict.irange_key(id, id):
            apartment_all_geom.append(e)

    list_of_apartments.append(apartment_all_geom)

In [39]:
list_x = [1,2,3]
print(list_x + [4,])

[1, 2, 3, 4]


In [40]:
import os

# get files which passed visual inspection
fine_files = os.listdir("./datasets/dataset_walls_corrected_2/output/")
fine_ids = list()

for file_name in fine_files:
    fine_ids.append(int(file_name.split('.')[0]))


In [41]:
def entrance_touches_communication(list_of_elements):
     
    entrance_door = None
     
    for el in list_of_elements:
        if el['entity_subtype'] == "ENTRANCE_DOOR":
            entrance_door = el
            
    if not entrance_door:
        return False
            
    for el in list_of_elements:       
        if el != entrance_door and output_display[el['entity_subtype']] == colors_out['comunication']:
            if shapely.dwithin(entrance_door['polygon'], el['polygon'], 0.05):
                return True
    
    return False

def has_required_rooms_types(list_of_elements):
    
    colors_dict = {
        'sanitary':  False,
        'priv_space': False,
        'kitchen': False, # or dining (same color)
        'comunication': False,
        'public_space':  False
    }

    for el in list_of_elements:
        for key in colors_dict.keys():
            if output_display[el['entity_subtype']] == colors_out[key]:
                colors_dict[key] = True
                
    for key in colors_dict.keys():
        if not colors_dict[key]:
            return False
 
    return True

def check_if_geom_touches_geom(geom_to_check, type_of_geom, list_of_dict_to_check, element_to_explude):

    touches_geom = False
    exclude_list = ['AIR', 'VOID', 'LOGGIA', 'BALCONY', 'PATIO', 'OUTDOOR_VOID',
                    'ARCADE', 'BATHTUB', 'BUILT_IN_FURNITURE', 'BUILT_IN_FURNITURE',
                    'ELECTRICAL_SUPPLY', 'ELEVATOR_FACILITIES', 'GARDEN', 'GAS',
                    'HEATING', 'HOUSE_TECHNICS_FACILITIES', 'LIGHTWELL', 'OPERATIONS_FACILITIES',
                    'OIL_TANK', 'OUTDOOR_VOID', 'PATIO', 'RAILING', 'SHOWER', 'SINK',
                    'TERRACE', 'WASHING_MACHINE', 'WATER_SUPPLY']

    for element_to_check in list_of_dict_to_check:
        if (element_to_check != element_to_explude and element_to_check['entity_subtype'] not in exclude_list):

            if type_of_geom in ['ENTRANCE_DOOR', 'DOOR', 'WINDOW'] and element_to_check['entity_subtype'] in ['WALL', 'COLUMN']:
                break
            if type_of_geom in ['WALL', 'COLUMN'] and element_to_check['entity_subtype'] in ['ENTRANCE_DOOR', 'DOOR', 'WINDOW' ]:
                break

            touches_geom_check = shapely.dwithin(geom_to_check, element_to_check['polygon'], 0.01)
            if touches_geom_check:
                touches_geom = touches_geom_check
                break

    return touches_geom

            
def is_ortho(list_of_elements):
    
    first_ang = None
    
    for el in list_of_elements:        
        if el['entity_subtype'] in ['WALL']:
            
            if el['polygon'] is not None:
                
                wall_points = el['polygon'].exterior.coords[:-1] # deleted last duplicated with first point    
                
                if len(wall_points) == 4:                
                    x = wall_points[1][0] - wall_points[0][0]
                    y = wall_points[1][1] - wall_points[0][1]            
                    ang = np.arctan2(y, x) * 180 / np.pi
                    
                    if not first_ang:
                        first_ang = ang
                    
                    if 3.0 < abs(ang - first_ang) % 90.0 < 87:                        
                        print(abs(ang - first_ang))
                        return False
    
    return True


def sort_elements(list_of_elements):
    for e in list_of_elements:
        if e['entity_subtype'] == 'WALL':
            list_of_elements.append(list_of_elements.pop(list_of_elements.index(e))) # move to end
    for e in list_of_elements:
        if e['entity_subtype'] == 'WINDOW':
            list_of_elements.append(list_of_elements.pop(list_of_elements.index(e))) # move to end
    for e in list_of_elements:
        if e['entity_subtype'] == 'DOOR':
            list_of_elements.append(list_of_elements.pop(list_of_elements.index(e))) # move to end
    for e in list_of_elements:
        if e['entity_subtype'] == 'ENTRANCE_DOOR':
            list_of_elements.append(list_of_elements.pop(list_of_elements.index(e))) # move to end

def short_extended_walls(apartment_elements_list):

    for element in apartment_elements_list:
        if element['entity_subtype'] in ['WALL', 'DOOR', 'ENTRANCE_DOOR', 'COLUMN', 'WINDOW']:

            try:
                geom_disrupted = False       
                wall_points = element['polygon'].exterior.coords[:-1] # deleted last duplicated with first point 

                if len(wall_points) == 4:

                    d0 = shapely.distance(shapely.Point(wall_points[0]), shapely.Point(wall_points[1]))
                    d1 = shapely.distance(shapely.Point(wall_points[1]), shapely.Point(wall_points[2]))

                    if d0 < d1:
                        short_line_pts = [[wall_points[0], wall_points[1]], [wall_points[2], wall_points[3]]]
                    else:
                        short_line_pts = [[wall_points[1], wall_points[2]], [wall_points[3], wall_points[0]]]   
                        
                    for i in range(2):
                        short_line = shapely.LineString(short_line_pts[i])

                        touches_geom = check_if_geom_touches_geom(short_line, element['entity_subtype'], apartment_elements_list, element)                    

                        if not touches_geom and not geom_disrupted:
                            vector_p0 = short_line_pts[0][1]
                            vector_p1 = short_line_pts[1][0]

                            if vector_p0 in short_line_pts[i]:
                                vector_points = [vector_p0, vector_p1]                            
                            else:
                                vector_points = [vector_p1, vector_p0]            

                            distance = shapely.distance(shapely.Point(vector_points[1]), 
                                                        shapely.Point(vector_points[0]))
                            unit_vector = ((vector_points[1][0] - vector_points[0][0]) / distance,
                                        (vector_points[1][1] - vector_points[0][1]) / distance)                        
                            
                            move_step = 0.01
                            move_vector = [unit_vector[0] * move_step, unit_vector[1] * move_step]

                            moved_line = shapely.LineString(short_line)

                            jumps_num = 0
                            while not touches_geom and not geom_disrupted:
                                jumps_num += 1
                                moved_line = shapely.transform(moved_line, lambda p: p + move_vector)
                                touches_geom = check_if_geom_touches_geom(moved_line, element['entity_subtype'], apartment_elements_list, element)

                                if jumps_num * move_step >= max(d0,d1):
                                    geom_disrupted = True

                            short_line_pts[i] = list(zip(*moved_line.coords.xy))
                            
                    list_of_new_polygon_points = short_line_pts[0][:] 
                    list_of_new_polygon_points.append(short_line_pts[1][0])
                    list_of_new_polygon_points.append(short_line_pts[1][1])
                    list_of_new_polygon_points.append(short_line_pts[0][0])

                    if geom_disrupted:
                        element['polygon'] = None
                    else:
                        element['polygon'] = shapely.Polygon(list_of_new_polygon_points)  
                
                else:
                    # print("wrong num of points ({1}) on {0}".format(element['entity_subtype'], len(wall_points) + 1))
                    pass
            except Exception:
                print('error')



In [42]:
find max x and y sizes of dwelling units

max_w, max_h = 0, 0
size_list = list()

for apartment_elements in list_of_apartments[:]:

    x_min, x_max, y_min, y_max = 0, 0, 0, 0 
    w_apartment, h_apartment = 0, 0

    for idx_0, element in enumerate(apartment_elements):      
        
        if element['polygon'] != None:
            for idx_1, point in enumerate(element['polygon'].exterior.coords):
                point_x = point[0]
                point_y = point[1]
                
                if idx_0 == 0 and idx_1 == 0:
                    x_min, x_max = point_x, point_x
                    y_min, y_max = point_y,point_y

                if point_x > x_max:
                    x_max = point_x
                if point_y > y_max:
                    y_max = point_y         
                if point_y < y_min:
                    y_min = point_y
                if point_x < x_min:
                    x_min = point_x

    w_apartment = x_max - x_min
    h_apartment = y_max - y_min

    if w_apartment > max_w:
        max_w = w_apartment
    if h_apartment > max_h:
        max_h = h_apartment
    
    size_list.append(w_apartment if w_apartment > h_apartment else h_apartment)

print(max_w, max_h)


In [43]:
fig, ax = plt.subplots()

ax.hist(size_list, bins = 100)

plt.show()

In [44]:
image_size =  256
max_size =  25.0 # max_w if max_w > max_h else max_h
scale_coef = image_size / (max_size * 1.0)

print(scale_coef)

10.24


In [45]:
FOLDER_NAME = "dataset_fine_256"

In [46]:
image_num = 0
apartment_num = len(list_of_apartments)
batch_size = 1000
round = 0

while round * batch_size < apartment_num:

    images_in = list()
    images_out = list()
    images_random = list()

    range_start = round * batch_size
    range_end = (round + 1) * batch_size if (round + 1) * batch_size < apartment_num else apartment_num

    round += 1

    for apartment_elements in list_of_apartments[range_start:range_end]:
               
        # clear walls extensions
        short_extended_walls(apartment_elements)

        x_min, x_max, y_min, y_max = 0, 0, 0, 0 
        w_apartment, h_apartment = 0, 0

        for idx_0, element in enumerate(apartment_elements): 

            if element['polygon'] != None:
                for idx_1, point in enumerate(element['polygon'].exterior.coords):
                    point_x = point[0]
                    point_y = point[1]
                    
                    if idx_0 == 0 and idx_1 == 0:
                        x_min, x_max = point_x, point_x
                        y_min, y_max = point_y,point_y

                    if point_x > x_max:
                        x_max = point_x
                    if point_y > y_max:
                        y_max = point_y         
                    if point_y < y_min:
                        y_min = point_y
                    if point_x < x_min:
                        x_min = point_x

        w_apartment = x_max - x_min
        h_apartment = y_max - y_min

        if w_apartment <= max_size and h_apartment <= max_size:

            image_num += 1
            
            if image_num % 100 == 0:
                print(image_num)          
    
            if not image_num in fine_ids:
                print('not in num')
                continue
                
            if not has_required_rooms_types(apartment_elements):          
                print('no has_required_rooms_types')
                continue
                
            if not is_ortho(apartment_elements):   
                print('no is_ortho')
                continue
                
            if not entrance_touches_communication(apartment_elements):  
                print('no entrance_touches_communication')
                continue

            im_in = Image.new("RGB", (image_size, image_size), color = (255, 255, 255))    
            im_out = Image.new("RGB", (image_size, image_size), color = (255, 255, 255))    
            im_random = Image.new("RGB", (image_size, image_size), color = (255, 255, 255))    

            draw_in = ImageDraw.Draw(im_in)
            draw_out = ImageDraw.Draw(im_out)
            draw_random = ImageDraw.Draw(im_random)


            for idx_0, element in enumerate(apartment_elements):
                if element['polygon'] != None and  element['entity_subtype']  == 'WALL':
                    first_wall = element['polygon']           

            two_points = first_wall.exterior.coords[:2]
            angle = np.arctan2(two_points[0][0] - two_points[1][0], two_points[0][1] - two_points[1][1])
            angle = np.degrees(angle) if angle >= 0 else np.degrees(angle) + 360

            # sort elements for proper draw order reason 
            sort_elements(apartment_elements)

            for idx_0, element in enumerate(apartment_elements):      
                
                if element['polygon'] != None:
                    polygon = element['polygon']

                    # if element['entity_subtype'] not in ['DOOR', 'ENTRANCE_DOOR']:
                    #     continue
                        
                    matrix_01 = [1, 0, 0, 1, -x_min, -y_min] # translate to 0,0
                    polygon_trans = shapely.affinity.affine_transform(polygon, matrix_01)
                
                    polygon_trans = shapely.affinity.scale(polygon_trans, xfact=scale_coef, yfact=scale_coef, origin=shapely.Point(0,0)) # scale by factor
                
                    matrix_02 = [1, 0, 0, 1, (image_size / 2) - (w_apartment * scale_coef / 2) ,(image_size / 2) -  (h_apartment * scale_coef / 2)] # translate to center   
                    polygon_trans = shapely.affinity.affine_transform(polygon_trans, matrix_02)

                    polygon_trans = shapely.affinity.rotate(polygon_trans, angle, origin=shapely.Point((image_size / 2),(image_size / 2)), use_radians=False)
                    
                    points = polygon_trans.exterior.coords


                    # color = (random.randrange(0, 255, 8),random.randrange(0, 255, 8),random.randrange(0, 255, 8))
                    out_disp_temp = output_display[element['entity_subtype']]
                    if out_disp_temp != None:        
                        draw_out.polygon(points[:], fill=out_disp_temp)

                    in_disp_temp = input_display[element['entity_subtype']]
                    if in_disp_temp != None:        
                        draw_in.polygon(points[:], fill=in_disp_temp)
            
                    draw_random.polygon(points[:], fill=(random.randrange(0, 255, 8), random.randrange(0, 255, 8), random.randrange(0, 255, 8)))

            im_in.save(f"datasets/{FOLDER_NAME}/input/{image_num}.png" ,"PNG")
            im_out.save(f"datasets/{FOLDER_NAME}/output/{image_num}.png" ,"PNG")

            images_in.append(im_in)
            images_out.append(im_out)
            images_random.append(im_random)

print(image_num)

no has_required_rooms_types
no entrance_touches_communication
no has_required_rooms_types
no has_required_rooms_types
52.668449796151776
no is_ortho
no has_required_rooms_types
no has_required_rooms_types
no has_required_rooms_types
no has_required_rooms_types
191.00000000000006
no is_ortho
not in num
no has_required_rooms_types
not in num
no has_required_rooms_types
no has_required_rooms_types
40.25898060189681
no is_ortho
no has_required_rooms_types
86.52418898560518
no is_ortho
no has_required_rooms_types
314.61948099046356
no is_ortho
no has_required_rooms_types
no has_required_rooms_types
no has_required_rooms_types
not in num
no has_required_rooms_types
30.825014630624892
no is_ortho
no has_required_rooms_types
no has_required_rooms_types
46.20660137667318
no is_ortho
not in num
150.1291650680306
no is_ortho
no has_required_rooms_types
no has_required_rooms_types
97.42331976591777
no is_ortho
34.53845347814793
no is_ortho
112.16913347127084
no is_ortho
no has_required_rooms_types

In [ ]:
print(len(images_in))

In [ ]:
# for i, im_in in enumerate(images_in):
#     im_in.save(f"dataset/input/{i}.png" ,"PNG")
 
# for i, im_out in enumerate(images_out):
#     im_out.save(f"dataset/output/{i}.png" ,"PNG")


In [ ]:
offset = 150
rows = 5
fig, axs = plt.subplots(ncols=3, nrows=rows, figsize=(3 * 8, rows * 8))

for idx in range(rows):
    axs[idx, 0].imshow(images_in[idx + offset])
    axs[idx, 1].imshow(images_out[idx + offset])
    axs[idx, 2].imshow(images_random[idx + offset])


In [ ]:
fig, axs = plt.subplots(ncols=1, nrows=11, figsize=(5, 5))

for idx, key in enumerate(colors_out):
    axs[idx].annotate(key, (0.5, 0.5), ha='center', va='center', fontsize=12, color='black')
    backgraund_color = (colors_out[key][0] / 255.0, colors_out[key][1] / 255.0, colors_out[key][2] / 255.0)
    axs[idx].set_facecolor(backgraund_color)
    
    axs[idx].set_axis_off()
    axs[idx].add_artist(axs[idx].patch)
    axs[idx].patch.set_zorder(-1)